In [1]:
# Load data

import numpy as np
import pandas as pd
import altair as alt

df_skaters = pd.read_csv('C:/Users/Admin/Desktop/Career/Projects/NHL Project/Project/data/game_skater_stats.csv')

In [2]:
df_skaters.head()

,game_id,player_id,team_id,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,...,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,shortHandedTimeOnIce,powerPlayTimeOnIce
0,2016020045,8468513,4,955,1,0,0,2.0,0,0,...,0,1.0,1.0,0,0,1.0,1,858,97,0
1,2016020045,8476906,4,1396,1,0,4,2.0,0,0,...,0,1.0,2.0,0,0,2.0,0,1177,0,219
2,2016020045,8474668,4,915,0,0,1,1.0,0,0,...,0,2.0,0.0,0,0,0.0,-1,805,0,110
3,2016020045,8473512,4,1367,3,0,0,0.0,0,2,...,27,0.0,0.0,0,0,0.0,-1,1083,19,265
4,2016020045,8471762,4,676,0,0,3,2.0,0,0,...,0,0.0,1.0,0,0,0.0,-1,613,63,0


In [6]:
# Merge with player data

df_players = pd.read_csv('C:/Users/Admin/Desktop/Career/Projects/NHL Project/Project/data/player_info.csv')

df_skaters = df_skaters.merge(df_players, left_on='player_id', right_on='player_id')

In [9]:
# Drop duplicate rows

df_skaters = df_skaters.drop_duplicates()

In [10]:
# Replace NAs with 0s

df_skaters = df_skaters.fillna(0)

In [13]:
# Drop goalies

df_skaters = df_skaters[df_skaters['primaryPosition'] != 'G']

In [15]:
# Convert to binary outcome

def map_position(primary_position):
    if primary_position in ['LW', 'RW', 'C']:
        return 0
    elif primary_position == 'D':
        return 1
    else:
        return 'unknown'

df_skaters['position'] = df_skaters['primaryPosition'].apply(map_position)

In [26]:
df_skaters.head()

,game_id,player_id,team_id,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,...,nationality,birthCity,primaryPosition,birthDate,birthStateProvince,height,height_cm,weight,shootsCatches,position
0,2016020045,8468513,4,955,1,0,0,2.0,0,0,...,CAN,Strasbourg,D,1982-08-25 01:00:00,SK,"6' 1""",185.42,203.0,L,1
1,2015020906,8468513,4,1148,0,0,1,0.0,0,0,...,CAN,Strasbourg,D,1982-08-25 01:00:00,SK,"6' 1""",185.42,203.0,L,1
2,2016020070,8468513,4,1184,1,0,1,1.0,0,0,...,CAN,Strasbourg,D,1982-08-25 01:00:00,SK,"6' 1""",185.42,203.0,L,1
3,2015020007,8468513,4,1096,0,0,0,1.0,0,0,...,CAN,Strasbourg,D,1982-08-25 01:00:00,SK,"6' 1""",185.42,203.0,L,1
4,2016020759,8468513,4,816,0,0,1,0.0,0,0,...,CAN,Strasbourg,D,1982-08-25 01:00:00,SK,"6' 1""",185.42,203.0,L,1


In [27]:
# Create 'points' from goals and assists

df_skaters['points'] = df_skaters['goals'] + df_skaters['assists']

In [40]:
# Create data frames for player stats and aggregate stats and combine them into a new data frame

df_skaters_player = df_skaters[['player_id', 'position', 'weight']]
df_skaters_player = df_skaters_player.groupby(by='player_id').mean()

df_skaters_game = df_skaters[['player_id', 'hits', 'points']]
df_skaters_game = df_skaters_game.groupby(by='player_id').sum()

df_skaters_total = df_skaters_player.merge(df_skaters_game, left_on='player_id', right_on='player_id')

In [56]:
df_skaters_total.head()

,position,weight,hits,points
player_id,,,,
8444894,0.0,196.0,0.0,23
8444919,1.0,195.0,0.0,41
8445000,0.0,225.0,0.0,162
8445176,0.0,191.0,0.0,141
8445266,1.0,236.0,0.0,30


In [44]:
# Train-Test Split

from sklearn.model_selection import train_test_split

X = df_skaters_total[['position', 'weight', 'hits']]
y = df_skaters_total['points']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [52]:
# Build simple regression tree model

from sklearn import tree
from sklearn.model_selection import GridSearchCV

clf = tree.DecisionTreeRegressor()

# Define the grid values for hyperparameters
param_grid = {
    'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10],
    'max_features': [None, 'sqrt', 'log2']
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

# Show the best parameters
best_params = grid_search.best_params_
best_params

# Train the model with the best parameters
best_clf = grid_search.best_estimator_

In [60]:
# Make predictions

y_pred = best_clf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mse

24273.7028435196

As expected, this is a TERRIBLE model. I could spend time trying to improve it, but I think that's a mistake because a) the feature variables on their own have poor predictive power (weight, position, and hits are horribly confounded and don't tell you much on their own) and b) a regression tree is a poor model choice in the first place.